In [24]:
class ReverseCalkinWilfNode:
    root_numerator, root_denominator = 1, 1

    @classmethod
    def path_to_root(cls, fraction_tuple):
        N = cls([], fraction_tuple[0], fraction_tuple[1])
        P = N.P()
        while True:
            if P.numerator == cls.root_numerator and P.denominator == cls.root_denominator:
                return P
            else:
                P = P.P()

    def __init__(self, move_list, numerator, denominator):
        self.move_list = move_list
        self.numerator = numerator
        self.denominator = denominator

    def P(self):
        if self.numerator < self.denominator:
            new_move_list = ["L"] + self.move_list
            new_numerator = self.numerator
            new_denominator = self.denominator - self.numerator
            return type(self)(new_move_list, new_numerator, new_denominator)
        elif self.numerator > self.denominator:
            new_move_list = ["R"] + self.move_list
            new_numerator = self.numerator - self.denominator
            new_denominator = self.denominator
            return type(self)(new_move_list, new_numerator, new_denominator)

    def __repr__(self):
        return f"{type(self).__name__}({self.move_list}, {self.numerator}, {self.denominator})"


if __name__ == "__main__":
    print(ReverseCalkinWilfNode.path_to_root((3, 5)))

ReverseCalkinWilfNode(['L', 'R', 'L'], 1, 1)


In [26]:
import itertools as it

class CalkinWilfNode:
    root_level = 0
    root_level_index = 0
    root_move_list = []
    root_fraction_tuple = (1, 1)

    @staticmethod
    def bfs_binary_string(n: int):
        binary_string = f"{n:b}"
        return [(k, len(list(g))) for k, g in it.groupby(reversed(binary_string))]

    @classmethod
    def locate_fraction_tuple(cls, fraction_tuple):
        move_list = ReverseCalkinWilfNode.path_to_root(fraction_tuple).move_list
        return cls(-1, -1, move_list, fraction_tuple)

    def __init__(self, level=None, level_index=None, move_list=None, fraction_tuple=None):
        if level is None:
            self.level = type(self).root_level
            self.level_index = type(self).root_level_index
            self.move_list = type(self).root_move_list
            self.fraction_tuple = type(self).root_fraction_tuple
        else:
            self.level = level
            self.level_index = level_index
            self.move_list = move_list
            self.fraction_tuple = fraction_tuple
    
    @property
    def L(self):
        new_level = self.level + 1
        new_level_index = 2 * self.level_index
        new_move_list = self.move_list + ["L"]
        a, b = self.fraction_tuple
        new_fraction_tuple = (a, a + b)
        return type(self)(new_level, new_level_index, new_move_list, new_fraction_tuple)
    
    @property
    def R(self):
        new_level = self.level + 1
        new_level_index = 2 * self.level_index + 1
        new_move_list = self.move_list + ["R"]
        a, b = self.fraction_tuple
        new_fraction_tuple = (a + b, b)
        return type(self)(new_level, new_level_index, new_move_list, new_fraction_tuple)
    
    def run_list(self):
        return [(k, len(list(g))) for k, g in it.groupby(self.move_list)]
    
    def __repr__(self):
        return f"{type(self).__name__}({self.level}, {self.level_index}, {self.move_list}, {self.fraction_tuple})"

    
if __name__ == "__main__":
    N0 = CalkinWilfNode()
    N1 = N0.L
    print(N1)
    N2 = N1.R
    print(N2.R)

    print(CalkinWilfNode.locate_fraction_tuple((5, 2)))

CalkinWilfNode(1, 0, ['L'], (1, 2))
CalkinWilfNode(3, 3, ['L', 'R', 'R'], (5, 2))
CalkinWilfNode(-1, -1, ['L', 'R', 'R'], (5, 2))
